In [2]:
import websocket
import hmac
import hashlib
import base64
import json
import requests
import pandas as pd
import urllib.parse
import datetime as dt

from datetime import datetime
from dotenv import dotenv_values
from collections import defaultdict

In [3]:
keys = dotenv_values(".env")

def get_kraken_signature(urlpath, data, secret):

    postdata = urllib.parse.urlencode(data)
    encoded = (str(data['nonce']) + postdata).encode()
    message = urlpath.encode() + hashlib.sha256(encoded).digest()

    mac = hmac.new(base64.b64decode(secret), message, hashlib.sha512)
    sigdigest = base64.b64encode(mac.digest())
    return sigdigest.decode()

api_sec = keys["KRAKEN_SECRET"]

data = {
    "nonce": str(int(dt.datetime.now().timestamp())), 
    "ordertype": "limit", 
    "pair": "XBTUSD",
    "price": 37500, 
    "type": "buy",
    "volume": 1.25
}

In [4]:
data = {
    "nonce": str(int(dt.datetime.now().timestamp())), 
}
endpoint = "/0/public/Ticker"
signature = get_kraken_signature(endpoint, data, api_sec)
url = keys["KRAKEN_BASE"] + endpoint

headers = {
    "accept": "application/json"
}
params = {
    "pair": "BTC/USD, ETH/BTC"
}

r = requests.get(url, headers=headers)

#### Parsing
- No Foreign Currencies
- No ETH
- No XXBT

In [ ]:
pricemap = defaultdict(dict)
btc_pairs = []
ticker = r.json()["result"]

BLACKLIST = ["USDC", "USDT", "GBP", "AUD", "JPY", "EUR", "ETH", "CAD"]

#Hardcoded BTC-USD
pricemap["BTC-USD"]["a"] = float(ticker["XXBTZUSD"]["a"][0])
pricemap["BTC-USD"]["b"] = float(ticker["XXBTZUSD"]["b"][0])
pricemap["BTC-USD"]["c"] = float(ticker["XXBTZUSD"]["c"][0])
#pricemap["USD-BTC"] = 1/float(ticker["XXBTZUSD"]["c"][0])

for pair in r.json()["result"]:
    isWanted = True
    for b in BLACKLIST:
        if b in pair:
            isWanted = False

    if (pair[-4:] == "XXBT" or pair[-4:] == "ZUSD") and (pair[0] == "X"):
        #TODO: change this to splice strings, orders would prob still work
        isWanted = False

        #exception: XTZ
        if pair[:3] == "XTZ":
            isWanted = True

    if float(ticker[pair]["c"][0]) == 0.0:
        # TODO: think about the case where this is only temporarily 0.0
        isWanted= False
    
    if isWanted:
        p0, p1 = pair[:-3], pair[-3:]
        if p0 == "XBT":
            p0 = "BTC"
        elif p1 == "XBT":
            p1 = "BTC"
            btc_pairs.append(f"{p0}-{p1}")

        pricemap[f"{p0}-{p1}"]["a"] = float(ticker[pair]["a"][0])
        pricemap[f"{p0}-{p1}"]["b"] = float(ticker[pair]["b"][0])
        pricemap[f"{p0}-{p1}"]["c"] = float(ticker[pair]["c"][0])
        '''
        try:
            pricemap[f"{p1}-{p0}"] = 1.0/float(ticker[pair]["c"][0])
        except:
            pricemap[f"{p1}-{p0}"] = 0.0
        '''


pricemap

In [7]:
TAKER_FEE = 0.0026
MAKER_FEE = 0.0016

def get_fee_tier(volume):
    if volume >= 50000:
        return 0.0014, 0.0024
    else:
        return 0.0016, 0.0026

In [8]:
def compute_trade_result(inv_in, alt, dir=0):
    fees = 0
    inv_out = inv_in
    prev_curr = None

    triplets = [
        [[f"BTC-USD", MAKER_FEE, "buy"], [f"{alt}-BTC", MAKER_FEE, "buy"], [f"{alt}-USD", MAKER_FEE, "sell"]],
        [[f"{alt}-USD", MAKER_FEE, "buy"], [f"{alt}-BTC", MAKER_FEE, "sell"], [f"BTC-USD", MAKER_FEE, "sell"]],
    ]

    for t in triplets[dir]:
        fee = inv_out * t[1]

        p0, p1 = t[0].strip().split("-")
        if t[2] == "buy":
            prev_curr = p1
            inv_out *= 1.0/pricemap[t[0]]["a"]
        elif t[2] == "sell":
            prev_curr = p0
            inv_out *= pricemap[t[0]]["b"]

        #convert fee to usd and add to total
        if prev_curr != "USD":
            if t[2] == "buy":
                fees += fee * pricemap[f"{prev_curr}-USD"]["a"]
            elif t[2] == "sell":
                fees += fee * pricemap[f"{prev_curr}-USD"]["b"]
        else:
            fees += fee

    subtotal = inv_out
    total = subtotal - fees
    profit = total - inv_in

    return subtotal, total, profit, fees


In [9]:
inv_in = 10.0
TAKER_FEE, MAKER_FEE = get_fee_tier(inv_in)

data = defaultdict(list)

for pair in btc_pairs:
    p0, _ = pair.strip().split("-")
    #test both direction of each triplet
    #for triplet in [f"USD-BTC-{pid0}", f"USD-{pid0}-BTC"]:
    for dir in range(2):
        subtotal, total, profit, fees = compute_trade_result(inv_in, p0, dir)

        if dir == 0:
            data["triplet"].append(f"USD-BTC-{p0}")
        elif dir == 1:
            data["triplet"].append(f"USD-{p0}-BTC")

        data["total"].append(total)
        data["subtotal"].append(subtotal)
        data["profit"].append(profit)
        data["fees"].append(fees)

pd.set_option('display.max_rows', 200)
df = pd.DataFrame(
    data={
        "Triplet": data["triplet"],
        "Subtotal": data["subtotal"],
        "Total": data["total"],
        "Fees": data["fees"],
        "Profit": data["profit"],
        }).sort_values(by="Profit", ascending=False)

df

,Triplet,Subtotal,Total,Fees,Profit
65,USD-MINA-BTC,10.015903,9.937879,0.078024,-0.062121
110,USD-BTC-WBTC,10.007945,9.929925,0.078021,-0.070075
113,USD-XTZ-BTC,10.005293,9.927293,0.077999,-0.072707
95,USD-SOL-BTC,10.000869,9.922869,0.078000,-0.077131
53,USD-KSM-BTC,9.998007,9.920029,0.077978,-0.079971
55,USD-LINK-BTC,9.997980,9.919993,0.077987,-0.080007
105,USD-TRX-BTC,9.997923,9.919929,0.077994,-0.080071
91,USD-SC-BTC,9.997375,9.919420,0.077955,-0.080580
27,USD-DOT-BTC,9.996918,9.918926,0.077992,-0.081074
47,USD-KAVA-BTC,9.996454,9.918466,0.077988,-0.081534


### Experiment: Negative profit margin turned positive with adjusted limit order

In [10]:
#USD-BTC-NYM
fees = 0.0
initial = 10.0
limit0 = -0.35
limit2 = 0.0
final = initial

fees += initial * MAKER_FEE
final *= 1.0/(pricemap["SOL-USD"]["a"] + limit0)

fees += final * TAKER_FEE * (pricemap["SOL-USD"]["a"] + limit0)
final *= pricemap["SOL-BTC"]["b"]

fees += final * TAKER_FEE * (pricemap["BTC-USD"]["b"] + limit2)
final *= (pricemap["BTC-USD"]["b"] + limit2)

print(f"Subtotal: {final}")
print(f"Total: {final-fees}")
print(f"Fees: {fees}")

Subtotal: 10.03342366815476
Total: 9.975370190285712
Fees: 0.05805347786904762
